<a href="https://colab.research.google.com/github/cobrien87/work/blob/main/sql_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





Code below taken from **mini-lesson #3**. Reads in PerioadAtt bio and scanTimes data: 

In [ ]:
import pandas as pd
import sqlite3

def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:
    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()


def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:  
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

    #Bio TABLE
# Step 1: Read the csv file into a dataframe
input_df = pd.read_csv('sample_data/bio.csv')
 
# Step 2: Upload the dataframe to a SQL Table
pd_to_sqlDB(input_df,
            table_name='Bio',
            db_name='default.db')

#Scan TABLE
# Step 1: Read the csv file into a dataframe
input_df = pd.read_csv('sample_data/scanTimes.csv')
 
# Step 2: Upload the dataframe to a SQL Table
pd_to_sqlDB(input_df,
            table_name='Scan',
            db_name='default.db')

#Period Attendance TABLE
# Step 1: Read the csv file into a dataframe
input_df = pd.read_csv('sample_data/periodAttendance.csv')
 
# Step 2: Upload the dataframe to a SQL Table
pd_to_sqlDB(input_df,
            table_name='periodAtt',
            db_name='default.db')


2022-03-13 19:12:53 INFO: SQL DB default.db created
2022-03-13 19:12:53 INFO: SQL Table Bio created with 6 columns
2022-03-13 19:12:53 INFO: 1000 rows uploaded to Bio
2022-03-13 19:12:53 INFO: SQL DB default.db created
2022-03-13 19:12:53 INFO: SQL Table Scan created with 6 columns
2022-03-13 19:12:53 INFO: 3370 rows uploaded to Scan
2022-03-13 19:12:53 INFO: SQL DB default.db created
2022-03-13 19:12:53 INFO: SQL Table periodAtt created with 9 columns
2022-03-13 19:12:54 INFO: 26332 rows uploaded to periodAtt


 Write a SQL join query that retrieves from the tables shared with you the records of all of the unexcused period attendance absences in the school for the week of  1-24-22 sorted by student last name ascending.  You will use the resulting table of results, which you can call allCuts, in the async assignment.  Consider a cut to be any instance of a student scanning into the building and being marked absent in a class.  You will retrieve only the first name, last name, student ID, grade, scanTime, status, date, courseSection, attendance, period, and teacher.



In [ ]:
sql_query_string = """
    
    SELECT s.*,p.date, p.CourseSection, p.Attendance, p.Teacher, p.Period
    FROM scan as s
    INNER JOIN periodAtt as p
    on substr(s.scantime,1,instr(s.scantime,' ')-1)=p.date AND s.studentid=p.studentid
    WHERE p.Attendance='A'
    ORDER BY s.last ASC

   
    
"""
 
#Execute the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,First,Last,StudentID,Grade,ScanTime,Status,Date,CourseSection,Attendance,Teacher,Period
0,Barbra,Adkin,5053664,10,1/28/2022 7:56,Present on time,1/28/2022,MR21-06,A,"Boudrie, P",6
1,Azalee,Aja,3413346,11,1/26/2022 8:29,Tardy,1/26/2022,FS63-03,A,"Taghon, H",4
2,Cammy,Albares,6371735,11,1/28/2022 8:29,Tardy,1/28/2022,MC21-02,A,"Siena, V",2
3,Denny,Amazan,2260826,10,1/26/2022 8:02,Present on time,1/26/2022,FS61-01,A,"Bonalumi, K",2
4,Denny,Amazan,2260826,10,1/28/2022 8:01,Present on time,1/28/2022,FS61-01,A,"Bonalumi, K",2
...,...,...,...,...,...,...,...,...,...,...,...
922,Yong,Yestramski,2466438,12,1/26/2022 7:00,Present on time,1/26/2022,AL11-02,A,"Sonier, A",8
923,Dierdre,Yum,6062256,12,1/28/2022 7:18,Present on time,1/28/2022,HV11-03,A,"Agar, S",7
924,Dierdre,Yum,6062256,12,1/28/2022 7:18,Present on time,1/28/2022,EE87-02,A,"Poarch, A",6
925,Serina,Zagen,1212212,12,1/24/2022 9:16,Tardy,1/24/2022,HV11-01,A,"Costeira, R",1


**SQL Challenge 2 (Async assignment, also posted to slack.**
Use the allCuts table and the biographical table to retrieve a list of student cuts with all information from both tables with no duplication of fields.

In [ ]:
sql_query_string = """
    SELECT Allcuts.*, b.parent1phone, b.parent2phone, b.parentemail
    FROM (
    SELECT s.*,p.date, p.CourseSection, p.Attendance, p.Teacher, p.Period
    FROM scan as s
    INNER JOIN periodAtt as p
    on substr(s.scantime,1,instr(s.scantime,' ')-1)=p.date AND s.studentid=p.studentid
    WHERE p.Attendance='A'
    ORDER BY s.last ASC
    )
    AS Allcuts
    LEFT JOIN bio as b
    ON Allcuts.studentid=b.studentid
    ORDER BY Allcuts.teacher ASC
   
    
"""
 
#Execute the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,First,Last,StudentID,Grade,ScanTime,Status,date,CourseSection,Attendance,Teacher,Period,Parent1Phone,Parent2Phone,ParentEmail
0,Elfriede,Bakeley,6045452,12,1/26/2022 8:20,Present on time,1/26/2022,HV1X-03,A,"Agar, S",8,506-404-8558,506-956-6048,ebakeley@bakeley.com
1,Elfriede,Bakeley,6045452,12,1/28/2022 8:41,Present on time,1/28/2022,HV1X-03,A,"Agar, S",8,506-404-8558,506-956-6048,ebakeley@bakeley.com
2,Nickie,Balonek,6110252,12,1/27/2022 8:00,Present on time,1/27/2022,HV1X-04,A,"Agar, S",9,604-556-1080,604-408-3115,nickie.balonek@balonek.com
3,Glenn,Berray,4365772,12,1/26/2022 8:11,Tardy,1/26/2022,HV1X-03,A,"Agar, S",8,515-370-7348,515-372-1738,gberray@gmail.com
4,Asuncion,Berson,8254563,12,1/26/2022 7:43,Present on time,1/26/2022,HV1X-04,A,"Agar, S",9,204-318-1929,204-696-6138,aberson@aol.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,Fidelia,Vilven,7061034,9,1/24/2022 8:10,Tardy,1/24/2022,FS63-01,A,"Taghon, H",1,416-892-4321,416-209-3419,fvilven@vilven.org
923,Fidelia,Vilven,7061034,9,1/27/2022 8:03,Present on time,1/27/2022,FS63-01,A,"Taghon, H",1,416-892-4321,416-209-3419,fvilven@vilven.org
924,Fidelia,Vilven,7061034,9,1/28/2022 8:07,Present on time,1/28/2022,FS63-01,A,"Taghon, H",1,416-892-4321,416-209-3419,fvilven@vilven.org
925,Tammara,Wardrip,6847451,9,1/28/2022 8:02,Present on time,1/28/2022,FS63-01,A,"Taghon, H",1,650-803-1936,650-216-5075,twardrip@cox.net
